## Data integration script

The aim of this task is to integrate in the same file the following data:
- GPS;
- Weather;
- Waze;
- GTFS

Example of final file attributes (to use in ML algorithms):
* **route, gps_id, bus_code, gps_timestamp, gps_lat, gps_lon, trip_problem_code, *stop_id, scheduled_timestamp, scheduled_headway, actual_headway, precipitation, distance_gps_station, waze_attributes* **

Bulma output
* **trip_number/no_shape_code, route, shape_id/-, shape_sequence/-, shape_lat/-, shape_lon/-, gps_id, bus_code, gps_timestamp, gps_lat, gps_lon, distance_to_shape_point/-, threshold_distance_gps_shape/-, trip_problem_code, *stop_id, scheduled_timestamp, scheduled_headway, actual_headway, precipitation, distance_gps_station, waze_attributes* **

**TODO:** check if it is better to calculate the headway with relation to the first bus.

The task has the following steps:
### 1. Clean the data (removing missing/wrong data); OK
1.1 Separate Recife files and convert coordinates

1.2 Remove different cities of Waze

1.3 Remove Recife GPS data without route, lat or lon

### 2. Label shape file with route type (low/high frequency) OK
2.1 Calculate headway median of each route and headway median of the city

2.2 Update route.txt e shapes.csv

### 3. Update and run BULMA (Integration of GPS and GTFS); OK
3.1. Filter GPS data to common attributes: bus.code, latitude, longitude, timestamp, line.code, gps.id

After this, run for CG and Curitiba too.
### 4. Update and run BUSTE (interpolate stops timestamp) OK
### 5. Label each GPS with headway value and BB (headway, BB, id_bus_bb)
### 6. Label with precipitation (precipitation)

In [1]:
import utm
import os
import pandas as pd
from datetime import datetime
from numpy import median
import math

In [2]:
DELIMITER = ','
NEW_EMPTY_LINE = '\n'

### Data example

#### GPS
- bus_code
- timestamp
- route
- latitude
- longitude

#### Weather

#### Waze

#### GTFS

Pre-processing GPS data of Recife to separate files per day and to convert coordinates.

In [ ]:
def convertCoordinates(x, y):
    if (x == '-' or x == '0' or y == '-' or y == '0'):
        return ['-', '-']
    
    return utm.to_latlon(long(x), long(y), 25, 'M')
    
def separateGPSFilePerMonth(file_path):
    october_file = open(file_path + 'GPS_data_october.csv', 'w')
    november_file = open(file_path + 'GPS_data_november.csv', 'w')
    december_file = open(file_path + 'GPS_data_december.csv', 'w')
    columns_name = "Unidad" + DELIMITER +  "Instante" + DELIMITER + "Estado" + DELIMITER + "Comunica" + DELIMITER + "CoordX" + DELIMITER + "CoordY" + DELIMITER + "Linea" + DELIMITER + "Ruta" + DELIMITER + "Posicion" + DELIMITER + "Viaje" + DELIMITER + "Velocidad"
    october_file.write(columns_name + NEW_EMPTY_LINE)
    november_file.write(columns_name + NEW_EMPTY_LINE)
    december_file.write(columns_name + NEW_EMPTY_LINE)
    
    october = '2018-10'
    november = '2018-11'
    december = '2018-12'
    
    file_name = file_path + "GPS_data.csv"
    with open(file_name, 'r') as gps_data:
        next(gps_data)
        for line in gps_data:
            line_splitted = line.split(DELIMITER)
            date = line_splitted[1]
            lat = line_splitted[4]
            lon = line_splitted[5]
            new_coordinates = convertCoordinates(lat, lon)
            
            new_line = line_splitted[0] + DELIMITER + line_splitted[1] + DELIMITER + line_splitted[2] + DELIMITER + line_splitted[3] + DELIMITER + str(new_coordinates[0]) + DELIMITER + str(new_coordinates[1]) + DELIMITER + line_splitted[6] + DELIMITER + line_splitted[7] + DELIMITER + line_splitted[8] + DELIMITER + line_splitted[9] + DELIMITER + line_splitted[10]
            
            if (october in date):
                october_file.write(new_line)
            elif (november in date):
                november_file.write(new_line)
            elif (december in date):
                december_file.write(new_line)

def separateGPSFilePerDay(file_path):
    columns_name = "Unidad" + DELIMITER +  "Instante" + DELIMITER + "Estado" + DELIMITER + "Comunica" + DELIMITER + "CoordX" + DELIMITER + "CoordY" + DELIMITER + "Linea" + DELIMITER + "Ruta" + DELIMITER + "Posicion" + DELIMITER + "Viaje" + DELIMITER + "Velocidad"
    file_name = file_path + "GPS_data_october.csv"
    date_lines_dict = {}
    with open(file_name, 'r') as gps_data:
        next(gps_data)
        for line in gps_data:
            line_splitted = line.split(DELIMITER)
            date_time = line_splitted[1]
            date = date_time.split(' ')[0]
            
            if (date not in date_lines_dict):
                date_lines_dict[date] = []
            
            date_lines_dict[date].append(line)
            
    
    for key in date_lines_dict:
        new_file = open(file_path + 'GPS_data_' + key + '.csv', 'w')
        new_file.write(columns_name + NEW_EMPTY_LINE)
        
        for data_per_day in date_lines_dict[key]:
            new_file.write(data_per_day)
            
        new_file.close()
            
file_path = os.getcwd() + "/../data/input/Recife/GPS/"

separateGPSFilePerMonth(file_path)

separateGPSFilePerDay(file_path)

### 1. Clean the data
Removing missing/wrong data

#### 1.1 Waze

In [ ]:
# Waze
# Removing lines from different city

def removeDifferentCity(dir_name, city_label):
    
    for file_name in os.listdir(dir_name):
        
        if file_name.endswith(".csv"): # to get just files
            file_path = dir_name + file_name

            new_file = open(dir_name + 'clean/' + file_name, 'w')

            with open(file_path, 'r') as waze_data:
                new_file.write(next(waze_data)) # write the header
                
                for line in waze_data:
                    line_splitted = line.split(',')
                    city = line_splitted[1]

                    if (city_label in city):
                        new_file.write(line) # to add just lines of the city

                new_file.close()
            
# Alert: create the folder 'clean', set the path and the city name            
dir_path = os.getcwd() + "/../data/input/Recife/Waze/"
city = 'Recife'
removeDifferentCity(dir_path, city)

#### 1.2 GPS

Removing Recife data without *linea* (route) or *CoordX* or *CoordY*

In [ ]:
# GPS - Recife
# Removing lines without linea or CoordX or CoordY

def cleanRecifeGPS(dir_path):
    for file_name in os.listdir(dir_path):
        
        if file_name.endswith(".csv"): # to get just files
            file_path = dir_path + file_name
            
            new_file = open(dir_path + 'clean/' + file_name, 'w')

            with open(file_path, 'r') as gps_data:
                new_file.write(next(gps_data)) # write the header
                
                for line in gps_data:
                    line_splitted = line.split(',')
                    route = line_splitted[6]
                    lat = line_splitted[4]
                    lon = line_splitted[5]

                    if (route != '' and lat != '-' and lat != '' and lon != '-' and lon != ''):
                        new_file.write(line) # to add just lines not empty

                new_file.close()
                
# Alert: create the folder 'csv' 
dir_path = os.getcwd() + "/../data/input/Recife/GPS/"
cleanRecifeGPS(dir_path)

In [ ]:
# GPS - Curitiba
# Converting json file to csv file

# Alert: first, replace }{ by },{ 
# sed -i ':a;N;$!ba;s/}/},/g' *.json

def convertJSON2CSV(dir_path):
    columns_name = "bus_code" + DELIMITER + "lat" + DELIMITER + "lon" + DELIMITER + "timestamp" + DELIMITER + "route"
    
    for file_name in os.listdir(dir_path):
        
        if file_name.endswith(".json"): # to get just files
            file_path = dir_path + file_name
            
            new_file = open(dir_path + 'csv/' + file_name.split('.')[0] + '.csv', 'w')

            new_file.write(columns_name + NEW_EMPTY_LINE)
            
            with open(file_path, 'r') as gps_data:
                next(gps_data) # skip empty line 
                for line in gps_data:
                    line_splitted = line.split(',')
                    bus_code = line_splitted[0].split('":')[1]
                    lat = line_splitted[1].split('":')[1]
                    lon = line_splitted[2].split('":')[1]
                    timestamp = line_splitted[3].split('":')[1]
                    route = line_splitted[4].split('":')[1][:-1]
                    
                    new_line = bus_code + DELIMITER + lat + DELIMITER + lon + DELIMITER + timestamp + DELIMITER + route
                    
                    new_file.write(new_line + NEW_EMPTY_LINE)
            
dir_path = os.getcwd() + "/../data/input/Curitiba/GPS/"
convertJSON2CSV(dir_path)

### 2. Label shape file with route type

Label shape file with route type: high frequency or low frequency based on *headway median of the city (h_median)*.

- **High frequency:** headway mean of the route <= h_median
- **Low frequency:** headway mean of the route > h_median

<img src="trips_file_example.png">

The *stop_times* file has the information:
- Buses: all the schedules of the bus are grouped, e.g. the schedules of the first bus are the firsts lines (from 5:00 to 23:00)
- Headway: the diference of the *arrival_time* of the bus and the follow.

**Challenge: how to separate different buses to compare headways?**

In [5]:
# Alert: choose the city to label the gtfs

city = "CG"
# city = "Curitiba"
# city = "Recife"

In [ ]:
# Alert: Set the path with the city name            
dir_path = os.getcwd() + "/../data/input/" + city + "/GTFS/"
trips_file = 'trips.txt'
stop_times_file = 'stop_times.txt'

# Read trips.txt to get the route of the trips
def getRouteServiceTripsMap():
    route_service_trips_map = {}
    
    with open(dir_path + trips_file, 'r') as trips_data:
        next(trips_data) # skip header
        for line in trips_data:
            splitted_line = line.split(',')
            route = splitted_line[0]
            service_id = splitted_line[1]
            trip_id = splitted_line[2]
            
            if (route not in route_service_trips_map):
                route_service_trips_map[route] = {}
                
            if (service_id not in route_service_trips_map[route]):
                route_service_trips_map[route][service_id] = []
            
            route_service_trips_map[route][service_id].append(trip_id)
    
#     print(route_trips_map)
    
    return route_service_trips_map
    

# Read stop_times.txt to get the stop_times of each trip
# The output is: {route: {service_id: {stop_id: [time1, ... timen]}}} 
def getTripsStopsTimesMap(route_service_trips_map):
    route_service_stops_times = {}
    
    with open(dir_path + stop_times_file, 'r') as stops_times_data:
        next(stops_times_data) # skip header
        
        for line in stops_times_data:
            splitted_line = line.split(',')
            trip_id = splitted_line[0]
            stop_id = splitted_line[3]
            arrival_time = splitted_line[1]
            
            # get the route of the trip 
            for route_key, service_trips in route_service_trips_map.items():
                
                for service_key, trips in service_trips.items():
                    if trip_id in trips:
                        route = route_key
                        service = service_key
                        break

            if (route not in route_service_stops_times):
                route_service_stops_times[route] = {}
                
            if (service not in route_service_stops_times[route]):
                route_service_stops_times[route][service] = {}
            
            if (stop_id not in route_service_stops_times[route][service]):
                route_service_stops_times[route][service][stop_id] = []
            
            route_service_stops_times[route][service][stop_id].append(arrival_time)
                    
#     print(route_stops_times['34'])
    return route_service_stops_times


# Calculate the headway(difference between two timestamps) in seconds
def getHeadway(time1, time2):
    t1 = datetime.strptime(time1, "%H:%M:%S")
    t2 = datetime.strptime(time2, "%H:%M:%S")
    difference = t2 - t1
    
    return difference.seconds


In [ ]:
route_service_trips_map = getRouteServiceTripsMap()
route_service_stops_times = getTripsStopsTimesMap(route_service_trips_map)

In [ ]:
# Calculate headway median of each route and headway median of the city
# comparing times of the same stop, route and service
def calculateRoutesHeadways(route_service_stops_times):
    route_headway_median_map = {}
    sum_headways_city = []
    
    for route, services in route_service_stops_times.items():
        sum_headways_route = []

        for service, stopstimes in services.items():

            for stops, times in stopstimes.items():
                last_time = 0

                # Order times before to iterate
                ordered_times = sorted(times, key=lambda d: map(int, d.split(':')))
                print(route, stops, ordered_times)
                for time in ordered_times:

                    # skip the first timestamp or wrong data
                    if (last_time != 0 and not time.startswith('24:') and not last_time.startswith('24:') 
                        and not time.startswith('25:') and not last_time.startswith('25:') 
                        and not time.startswith('26:') and not last_time.startswith('26:')
                        and not time.startswith('27:') and not last_time.startswith('27:')
                        and not time.startswith('28:') and not last_time.startswith('28:')):
                        
                        # mean doesn't work because some routes run in specific times
                        headway = getHeadway(last_time, time)

                        sum_headways_route.append(headway)

                    last_time = time

        # calculate the median headway of the route
        if (len(sum_headways_route) > 0):
            headway_route = median(sum_headways_route)
            route_headway_median_map[route] = headway_route

            sum_headways_city.append(headway_route)
            
    median_headway_city = median(sum_headways_city)
    return (median_headway_city, route_headway_median_map)
    
median_headway_city, route_headway_median_map = calculateRoutesHeadways(route_service_stops_times)

In [ ]:
# Update routes.txt file with frequency (high or low) label
# based on its median headway and the median headway city
def updateRoutesFile(file_path, median_headway_city, route_headway_median_map):
    
    new_routes_file = open(file_path + 'routes_label.txt', 'w')
    
    with open(file_path + 'routes.txt', 'r') as routes_data:
        columns_name = next(routes_data).split(DELIMITER)
        
        new_routes_file.write(columns_name[0] + DELIMITER + columns_name[1] + DELIMITER + columns_name[2]
                               + DELIMITER + columns_name[3] + DELIMITER + columns_name[4] + DELIMITER 
                              + columns_name[5] + DELIMITER + columns_name[6] + DELIMITER + columns_name[7] 
                               + DELIMITER + 'frequency' + DELIMITER + columns_name[8]) # write header 
        
        for line in routes_data:
            line_splitted = line.split(DELIMITER)
            route = str(line_splitted[0])
            
            # When there was no data to calculate headway or when route headway > city headway
            label = 'low_frequency'
            headway = '-'
            
            if (route in route_headway_median_map):
                headway = route_headway_median_map[route]
                if (headway <= median_headway_city):
                    label = 'high_frequency'
            
            print('City: ' + city, 'Route: ' + route, 'headway_city: ' + str(median_headway_city), 
                  'headway_route: ' + str(headway), 'label: ' + label)
            
            new_line = line_splitted[0] + DELIMITER + line_splitted[1] + DELIMITER + line_splitted[2] + DELIMITER + line_splitted[3] + DELIMITER + line_splitted[4] + DELIMITER + line_splitted[5] + DELIMITER + line_splitted[6] + DELIMITER + line_splitted[7] + DELIMITER + label + DELIMITER + line_splitted[8]
            new_routes_file.write(new_line)
            
    new_routes_file.close()
    
    print("Saving routes_label.txt with route frequency.")

# Alert: change the city name
file_path = os.getcwd() + "/../data/input/" + city + "/GTFS/"
updateRoutesFile(file_path, median_headway_city, route_headway_median_map)

In [3]:
def readDataFromCSV(file_path):
    file_data = []
    
    with open(file_path, 'r') as data:
        columns_name = next(data) # skip header
        
        for line in data:
            file_data.append(line)
            
    return file_data


def saveData2CSV(file_path, new_data, header):
    new_file_data = open(file_path, 'w')
    new_file_data.write(header + NEW_EMPTY_LINE)
    
    for line in new_data:
        new_file_data.write(line + NEW_EMPTY_LINE)
        
    new_file_data.close()

In [13]:
# Add route id and frequency to shapes.txt file

file_path = os.getcwd() + "/../data/input/" + city + "/GTFS/"

def updateShapeFile(file_path):
    shape_path = file_path + "shapes.txt"
    trips_path = file_path + "trips.txt"
    routes_path = file_path + "routes_label.txt"
    
    new_shape_file = [];
    
    # Create a map {shapes: route} from trips.txt
    route_shape_map = {}
    trips_data = readDataFromCSV(trips_path)
    for trip in trips_data:
        attributes = trip.split(DELIMITER)
        route = attributes[0]
        
        if (city == "Recife"):
            shapeID = attributes[6][0:-2] # to remove "\r\n"
        else:
            shapeID = attributes[7][0:-1] # to remove "\n"
        
        if (shapeID not in route_shape_map):
            route_shape_map[shapeID] = route # assuming that a shape has one route
            
    # Create a map {route: frequency} from routes_label.txt
    route_frequency_map = {}
    routes_data = readDataFromCSV(routes_path)
    for routes in routes_data:
        attributes = routes.split(DELIMITER)
        route = attributes[0]
        frequency = attributes[8]
        
        if (city == "CG"): # the gps data route is route_short_name
            route_gps = attributes[2] # route_short_name
            
            if (route not in route_frequency_map):
                route_frequency_map[route] = frequency + "-" + route_gps
            
        else:
            if (route not in route_frequency_map):
                route_frequency_map[route] = frequency
    

    shapes_data = readDataFromCSV(shape_path)
    for shape in shapes_data:
        attributes = shape.split(DELIMITER)
        shapeID = attributes[0]
        
        route = "-"
        frequency = "-"
        if (shapeID in route_shape_map):
            route = route_shape_map[shapeID]
        
        other_attributes = shape[0:-1]
        if (city == "Recife"):
            other_attributes = shape[0:-2]
        
        new_line = route + DELIMITER + other_attributes + DELIMITER + frequency # for shape without route
        
        if (route in route_frequency_map):
            if (city == "CG"):
                frequency_route = route_frequency_map[route].split("-")
                frequency = frequency_route[0]
                route_id = frequency_route[1]
                shape_distance_traveled = float(attributes[4]) * 1000 # converting km to m
                
                other_attributes = attributes[0] + DELIMITER + attributes[1] + DELIMITER + attributes[2] + DELIMITER + attributes[3] + DELIMITER + str(shape_distance_traveled)
                
                new_line = route_id + DELIMITER + other_attributes + DELIMITER + frequency
                
            else:
                frequency = route_frequency_map[route]
                new_line = route + DELIMITER + other_attributes + DELIMITER + frequency
              
        new_shape_file.append(new_line)
        
    header = "route,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,route_frequency"
    
    saveData2CSV(file_path + "shapes1.csv", new_shape_file, header)
    print("Saving new shapes.csv with route id and frequency - " + city)

updateShapeFile(file_path)

Saving new shapes.csv with route id and frequency - CG


Each stop time belongs to one trip, i.e. the total os trips is the total os time stamps there.

**Observation:**

- In general, the scheduled headway of Curitiba is 1413 seconds (23 minutes).
- In general, the scheduled headway of Campina Grande is 1140.0 seconds (19 minutes).
- In general, the scheduled headway of Recife is 5115.0 seconds (1h42 minutes). (GTFS out of date)

### 3. Update and run BULMA/BUSTE (Integration between GPS and GTFS)

We need to use BULMA to correct the GPS position in the street and some cities has multiple shapes for the same route.

** City - Execution time - #Shapes per route **

Curitiba - 15 min (per file - day) - 2+

Campina Grande - 30s (per file - day) - 2+

Recife - 2 min (per file - day) - 2(+?)

Criar novo código Spark para:

After BULMA execution, add to the output:
- 3.6. The precipitation value (according to the hour and less distance)
- 3.7. The waze values (according to the same street and time)
- 3.8. The headway and BB value.

THE END OF THE INTEGRATION

In [1]:
# run 3.4, 3.5